In [ ]:
import pandas as pd

df = pd.read_csv("C:/projects/Ecommerce_Segmentation_Project/Data/cleaned_data.csv")
# -----------------------------
# 1. Calculate Snapshot Date
# -----------------------------
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
snapshot_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)

# -----------------------------
# 2. Create R, F, M values
# -----------------------------
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,   # Recency
    'InvoiceNo': 'nunique',                                   # Frequency
    'TotalPrice': 'sum'                                       # Monetary
})

# Rename columns
rfm.rename(columns={
    'InvoiceDate': 'Recency',
    'InvoiceNo': 'Frequency',
    'TotalPrice': 'Monetary'
}, inplace=True)

# -----------------------------
# 3. Create RFM Scores (1-5)
# -----------------------------
rfm['R_score'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1])
rfm['F_score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
rfm['M_score'] = pd.qcut(rfm['Monetary'], 5, labels=[1,2,3,4,5])

# Convert scores to string for segmentation logic
rfm['R_score'] = rfm['R_score'].astype(str)
rfm['F_score'] = rfm['F_score'].astype(str)
rfm['M_score'] = rfm['M_score'].astype(str)

# -----------------------------
# 4. Create RFM Score (2-digit combined)
# -----------------------------
rfm['RFM_Score'] = rfm['R_score'] + rfm['F_score']

# -----------------------------
# 5. Segmentation Function
# -----------------------------
def segment_customer(row):

    # Champions (recent + frequent)
    if row['R_score'] in ['4','5'] and row['F_score'] in ['4','5']:
        return 'Champions'
    
    # Loyal Customers
    if row['F_score'] in ['3','4','5'] and row['R_score'] in ['3','4','5']:
        return 'Loyal Customers'

    # Potential Loyalists
    if row['R_score'] in ['4','5'] and row['F_score'] in ['2','3']:
        return 'Potential Loyalists'
    
    # Recent Customers
    if row['R_score'] == '5' and row['F_score'] == '1':
        return 'Recent Customers'
    
    # Promising
    if row['R_score'] == '4' and row['F_score'] == '1':
        return 'Promising'
    
    # At Risk
    if row['R_score'] in ['1','2'] and row['F_score'] in ['3','4','5']:
        return 'At Risk'
    
    # Hibernating
    if row['R_score'] == '2' and row['F_score'] == '1':
        return 'Hibernating'
    
    # Lost
    if row['R_score'] == '1' and row['F_score'] == '1':
        return 'Lost'
    
    return 'Others'

# Apply segmentation
rfm['Segment'] = rfm.apply(segment_customer, axis=1)

# -----------------------------
# 6. Reset index for Power BI
# -----------------------------
rfm_reset = rfm.reset_index()

# -----------------------------
# 7. Save the final RFM CSV
# -----------------------------
rfm_reset.to_csv("C:/projects/Ecommerce_Segmentation_Project/Data/rfm_Final.csv", index=False)
